# 🧹 02 - Nettoyage des données OpenFoodFacts
Ce notebook a pour objectif de charger les données brutes transformées en Parquet et de nettoyer les colonnes inutiles, mal remplies ou trop bruitées.


In [1]:
# ⚙️ Initialisation Spark (si pas déjà fait comme en dehors du contexte JupyterLab)
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Cleaning OpenFoodFacts").getOrCreate()


In [2]:
# 📥 Chargement des données depuis le fichier Parquet (étape 1)
input_path = "../data/step1_raw.parquet"
df = spark.read.parquet(input_path)
print(f"✅ Données chargées : {df.count():,} lignes, {len(df.columns)} colonnes")


Py4JJavaError: An error occurred while calling o28.parquet.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:817)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1415)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1620)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:739)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2078)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2122)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:961)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:218)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:132)
	at scala.collection.immutable.List.map(Li

In [ ]:
# 🧾 Aperçu des colonnes initiales
for i, col_name in enumerate(df.columns, start=1):
    print(f"{i:02d}. {col_name}")


In [ ]:
# 🧼 Suppression des colonnes inutiles ou trop bruitées
colonnes_a_supprimer = [
    'image_url', 'image_small_url', 'image_ingredients_url', 'image_ingredients_small_url',
    'image_nutrition_url', 'image_nutrition_small_url',
    'created_t', 'last_modified_t', 'last_updated_t', 'last_image_t',
    'product_name', 'generic_name', 'creator', 'last_modified_by',
    'code', 'url', 'packaging_text', 'stores', 'purchase_places'
]

df_cleaned = df.drop(*[c for c in colonnes_a_supprimer if c in df.columns])
print(f"✅ Colonnes après nettoyage : {len(df_cleaned.columns)}")


In [ ]:
# 🔍 Filtrage des lignes avec peu d'informations utiles (optionnel)
from pyspark.sql.functions import col
df_filtered = df_cleaned.filter(col("ingredients_text").isNotNull())
print(f"✅ Lignes restantes avec ingrédients renseignés : {df_filtered.count():,}")


In [ ]:
# 💾 Sauvegarde des données nettoyées pour l'étape suivante
output_path = "../data/step2_cleaned.parquet"
df_filtered.write.mode("overwrite").parquet(output_path)
print(f"✅ Données nettoyées enregistrées dans : {output_path}")
